In [1]:
from pyspark.sql import SparkSession
from sparkmeasure import StageMetrics
from pyspark.sql.functions import lower, col

spark = SparkSession.builder.appName('q3').config("spark.jars", "/home/athina/Downloads/spark-measure_2.12-0.17.jar").getOrCreate()
#For VIRTUAL
#spark = SparkSession.builder.master('spark://picachu-VirtualBox:7077').appName('q3').config("spark.jars", "/home/athina/Downloads/spark-measure_2.12-0.17.jar").getOrCreate()

#For LIVY
#spark=SparkSession.builder.master('spark://localhost:7077').appName("Testing").getOrCreate()
#and changed the filepaths to '/home/administrator/Downloads/movielens/file_name.csv'
stagemetrics = StageMetrics(spark)
stagemetrics.begin()#start measuring performance

df_tag= (spark.read
     .format("csv")
     .option('header', 'true')
     .option('delimiter', ",")
     .option("inferSchema","true")
     .load("tag.csv") 
    )
df_rating= (spark.read
     .format("csv")
     .option('header', 'true')
     .option('delimiter', ",")
     .option("inferSchema","true")
     .load("rating.csv") 
    )

lc=df_tag.select('*',lower(col('tag')).alias('tag_lc'))#gets low case tags
#bollywood=lc.filter(lc.tag_lc.like("bollywood")).select("userId","movieId") # has only userdID & movieId with tag==bollywood
bollywood=lc.filter(lc.tag_lc.like("bollywood%")).select("userId","movieId") # has only userdID & movieId with tag==bollywood | bollywood influece

more_than_3=df_rating.filter(df_rating["rating"]>3).select("userId","movieId") # has only userdID & movieId with rating>3

# has only userId which have tag a movie as Bollywood and has rated the same movie more than 3 # the users are unique
results=more_than_3.join(bollywood, bollywood["movieId"]==more_than_3["movieId"],"inner").where(bollywood["userId"]==more_than_3["userId"]).select(bollywood["userId"]).hint("broadcast").distinct()
results.orderBy("userId").show(5)#requested answer for query 3

stagemetrics.end()#stop measuring performance
stagemetrics.print_report()#print performance metrics
spark.stop()

+------+
|userId|
+------+
| 10573|
| 19837|
| 23333|
| 25004|
| 31338|
+------+
only showing top 5 rows


Scheduling mode = FIFO
Spark Context default degree of parallelism = 2
Aggregated Spark stage metrics:
numStages => 7
numTasks => 218
elapsedTime => 31031 (31 s)
stageDuration => 29975 (30 s)
executorRunTime => 44777 (45 s)
executorCpuTime => 41802 (42 s)
executorDeserializeTime => 4407 (4 s)
executorDeserializeCpuTime => 3018 (3 s)
resultSerializationTime => 38 (38 ms)
jvmGCTime => 1033 (1 s)
shuffleFetchWaitTime => 4 (4 ms)
shuffleWriteTime => 65 (65 ms)
resultSize => 1325925 (1294.0 KB)
diskBytesSpilled => 0 (0 Bytes)
memoryBytesSpilled => 0 (0 Bytes)
peakExecutionMemory => 153092096
recordsRead => 33126961
bytesRead => 1425107724 (1359.0 MB)
recordsWritten => 0
bytesWritten => 0 (0 Bytes)
shuffleRecordsRead => 24
shuffleTotalBlocksFetched => 24
shuffleLocalBlocksFetched => 24
shuffleRemoteBlocksFetched => 0
shuffleTotalBytesRead => 1416 (1416 Bytes)
shuffleLocalBytesRead => 14